In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn import metrics
from sklearn.neural_network import MLPRegressor
from joblib import dump,load

In [2]:
#读数据标准化划分数据集
df = pd.read_csv('../my_data.csv')
X = df.iloc[:, 1:58]
Y = df.iloc[:, 58:]
Y=Y['SLME @ 5um']
names=df.columns
names=['SHAP value of '+x for x in names]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=10)
print("X_train's shape is", X_train.shape,"; y_train's shape is", y_train.shape)
print("X_test's shape is", X_test.shape,"; y_test's shape is",y_test.shape)
scaler = StandardScaler()
scaler.fit(X_train)
X_train_stand = scaler.transform(X_train)
X_test_stand = scaler.transform(X_test)

X_train's shape is (390, 57) ; y_train's shape is (390,)
X_test's shape is (98, 57) ; y_test's shape is (98,)


In [3]:
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],  # 隐藏层神经元数量和层数的组合
    'activation': ['relu', 'tanh'],  # 激活函数选项
    'solver': ['adam', 'sgd'],  # 优化器选项
    'alpha': [0.0001, 0.001, 0.01],  # 正则化参数选项
    'learning_rate': ['constant', 'adaptive'],  # 学习率策略选项
    'max_iter': [200, 300, 400],  # 最大迭代次数
    'batch_size': [16, 32, 64],  # 批量大小
    'momentum': [0.9, 0.95, 0.99],  # 动量参数
    'early_stopping': [True, False],  # 提前停止
    'n_iter_no_change': [5, 10, 15],  # 没有改变迭代次数
    'shuffle': [True, False],  # 是否打乱数据
}

In [4]:
ann = MLPRegressor(random_state=10)
model = GridSearchCV(estimator =ann,scoring='neg_root_mean_squared_error', 
                     param_grid = param_grid,cv=3, verbose=3,n_jobs =-1,return_train_score=True)
model.fit(X_train, y_train)
print(model.cv_results_)
dfresult=pd.DataFrame(model.cv_results_)
dfresult.to_csv('SLME_ANN13y32.csv')
print(dfresult)
print(model.best_params_)
print(model.scorer_)
print(model.best_score_)
bestmodel=model.best_estimator_

Fitting 3 folds for each of 31104 candidates, totalling 93312 fits


D:\App\Anaconda3\envs\test\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
2232 fits failed out of a total of 93312.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1944 fits failed with the following error:
Traceback (most recent call last):
  File "D:\App\Anaconda3\envs\test\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\App\Anaconda3\envs\test\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py", line 752, in fit
    return self._fit(X, y, incremental=False)
  File "D:\App\Anaconda3\envs\test\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py", line 435, in _fit
    increm

       mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0           1.283375      0.787529         0.006189        0.001884   
1           1.988252      0.114636         0.003180        0.000255   
2           1.955132      0.100465         0.003179        0.000245   
3           1.471471      0.167930         0.003844        0.000616   
4           2.161271      0.073122         0.004018        0.001098   
...              ...           ...              ...             ...   
31099       0.473759      0.032537         0.003523        0.000006   
31100       0.563441      0.041520         0.002839        0.001029   
31101       0.532473      0.037133         0.002171        0.000238   
31102       0.336597      0.032798         0.002338        0.000470   
31103       0.451531      0.033796         0.002334        0.000469   

      param_activation param_alpha param_batch_size param_early_stopping  \
0                 relu      0.0001               16                 Tru